# https://github.com/apache/hbase/blob/master/hbase-thrift/src/main/resources/org/apache/hadoop/hbase/thrift2/hbase.thrift

In [50]:
import thrift as t
import hbase_thrift.hbase as hb

from hbase_thrift.hbase import THBaseService
from hbase_thrift.hbase.ttypes import *

from thrift.transport import *
from thrift.protocol import *

# Prototyping 

In [2]:
socket = TSocket.TSocket("localhost", 9090)
transport = TTransport.TBufferedTransport(socket)
protocol = TBinaryProtocol.TBinaryProtocol(transport)
client = THBaseService.Client(protocol)

In [3]:
transport.open()
# Check Thrift Server Type
serverType = client.getThriftServerType()
if serverType != TThriftServerType.TWO:
    raise Exception("Mismatch between client and server, server type is %s" % serverType)

In [4]:
c1 = TColumnFamilyDescriptor(name=b'c1')
c2 = TColumnFamilyDescriptor(name=b'c2')
name = TTableName(qualifier=b'test_table_thriftv2')
tt = TTableDescriptor(tableName=name, columns=[c1, c2])

In [5]:
#client.createTable(tt, None)

In [22]:
client.getTableNamesByNamespace('')

NameError: name 'client' is not defined

In [7]:
transport.close()

# Functional client

In [1]:
class HbaseConnection:
    def __init__(self, address, port):
        self.address = address
        self.port = port
        
        self._socket = None
        self._client = None
        
    def _create_transport(self):
        if self._socket != None and self._socket.isOpen():
            try:
                self._socket.close()
            except:
                pass
        
        self._socket = TSocket.TSocket(self.address, self.port)
        self._transport = TTransport.TBufferedTransport(self._socket)
        self._protocol = TBinaryProtocol.TBinaryProtocol(self._transport)
        self._client = THBaseService.Client(self._protocol)
        self._transport.open()
        
    def __enter__(self):
        if self._client == None or not self._socket.isOpen():
            self._create_transport()
        
        return self._client
    
    def __exit__(self, _, __, ___):
        pass

In [2]:
hbase = HbaseConnection("datanode1", 9090)

In [ ]:
with hbase.

In [3]:
## Deletes all tables in databse (except for hbase internal tables)
with hbase as c:
    for table in filter(lambda x: x.ns != b'hbase', tables):
        if c.isTableEnabled(table):
            c.disableTable(table)
        c.deleteTable(table)

NameError: name 'TSocket' is not defined

In [67]:
with hbase as c:
    c1 = TColumnFamilyDescriptor(name=b'c1')
    c2 = TColumnFamilyDescriptor(name=b'c2')
    name = TTableName(qualifier=b'test_table_thriftv2')
    tt = TTableDescriptor(tableName=name, columns=[c1, c2])
    c.createTable(tt, None)

TIOError: TIOError(message='org.apache.hadoop.hbase.TableExistsException: test_table_thriftv2\n\tat org.apache.hadoop.hbase.master.procedure.CreateTableProcedure.prepareCreate(CreateTableProcedure.java:246)\n\tat org.apache.hadoop.hbase.master.procedure.CreateTableProcedure.executeFromState(CreateTableProcedure.java:88)\n\tat org.apache.hadoop.hbase.master.procedure.CreateTableProcedure.executeFromState(CreateTableProcedure.java:54)\n\tat org.apache.hadoop.hbase.procedure2.StateMachineProcedure.execute(StateMachineProcedure.java:191)\n\tat org.apache.hadoop.hbase.procedure2.Procedure.doExecute(Procedure.java:956)\n\tat org.apache.hadoop.hbase.procedure2.ProcedureExecutor.execProcedure(ProcedureExecutor.java:1665)\n\tat org.apache.hadoop.hbase.procedure2.ProcedureExecutor.executeProcedure(ProcedureExecutor.java:1412)\n\tat org.apache.hadoop.hbase.procedure2.ProcedureExecutor.access$1100(ProcedureExecutor.java:78)\n\tat org.apache.hadoop.hbase.procedure2.ProcedureExecutor$WorkerThread.ru

In [26]:
with hbase as c:
    tables = c.getTableNamesByNamespace('')
tables

[TTableName(ns=b'default', qualifier=b'test_table_thriftv2'),
 TTableName(ns=b'hbase', qualifier=b'meta'),
 TTableName(ns=b'hbase', qualifier=b'namespace')]

In [40]:
col1 = TColumnValue(family=b'c1', qualifier=b'prefix2', value=b'uhhhh')
put = TPut(row=b'hello row', columnValues=[col1])

with hbase as c:
    c.put(b'test_table_thriftv2', put)

In [44]:
with hbase as c:
    v = c.get(b'test_table_thriftv2', TGet(row=b'hello row'))
v

TResult(row=b'hello row', columnValues=[TColumnValue(family=b'c1', qualifier=b'prefix', value=b'antotha one', timestamp=1650205432804, tags=None, type=4), TColumnValue(family=b'c1', qualifier=b'prefix2', value=b'uhhhh', timestamp=1650206736685, tags=None, type=4)], stale=False, partial=False)

In [56]:
with hbase as c:
    col1 = TColumnValue(family=b'c1', qualifier=b'prefix2', value=b'uhhhh')
    put = TPut(row=b'anotha row', columnValues=[col1])
    
    col2 = TColumnValue(family=b'c2', qualifier=b'prefix4', value=b'adaaaaaa')
    put2 = TPut(row=b'not row3', columnValues=[col2])
    
    r = c.putMultiple(b'test_table_thriftv2', [put, put2])
r

In [57]:
with hbase as c:
    r = c.getMultiple(b'test_table_thriftv2', [TGet(row=b'hello row'), TGet(row=b'hello row3')])
r

[TResult(row=b'hello row', columnValues=[TColumnValue(family=b'c1', qualifier=b'prefix', value=b'antotha one', timestamp=1650205432804, tags=None, type=4), TColumnValue(family=b'c1', qualifier=b'prefix2', value=b'uhhhh', timestamp=1650206736685, tags=None, type=4)], stale=False, partial=False),
 TResult(row=b'hello row3', columnValues=[TColumnValue(family=b'c2', qualifier=b'prefix4', value=b'adaaaaaa', timestamp=1650206736685, tags=None, type=4)], stale=False, partial=False)]

In [61]:
# filterBytes: https://hbase.apache.org/devapidocs/src-html/org/apache/hadoop/hbase/thrift2/ThriftUtilities.html#line.617
# So it's a protobuf serialized something

with hbase as c:
    s = TScan(filterString=b'PrefixFilter(\'hello\')')
    #s = TScan(reversed = True)
    scanner = c.openScanner(b'test_table_thriftv2', tscan=s)
    
    readRows = None
    rows = []
    while readRows == None or len(readRows) == 1: 
        readRows = c.getScannerRows(scanner, numRows=1)
        rows.append(readRows)
    c.closeScanner(scanner)
    
rows

[[TResult(row=b'hello row', columnValues=[TColumnValue(family=b'c1', qualifier=b'prefix', value=b'antotha one', timestamp=1650205432804, tags=None, type=4), TColumnValue(family=b'c1', qualifier=b'prefix2', value=b'uhhhh', timestamp=1650206736685, tags=None, type=4)], stale=False, partial=False)],
 [TResult(row=b'hello row3', columnValues=[TColumnValue(family=b'c2', qualifier=b'prefix4', value=b'adaaaaaa', timestamp=1650206736685, tags=None, type=4)], stale=False, partial=False)],
 []]

In [63]:
with hbase as c:
    r = c.tableExists(TTableName(qualifier=b'nope'))
r

False

In [64]:
with hbase as c:
    r = c.tableExists(TTableName(qualifier=b'test_table_thriftv2'))
r

True

In [69]:
with hbase as c:
    r = c.exists(b'test_table_thriftv2', TGet(row=b'hello row'))
r

True